In [1]:
import numpy as np
import math
import pandas as pd
from kenglish.mrlattice import (
    generationAffineParamsFromPath, 
    mrTupFromPath, 
    mrTupValue
)

# Starting Over with Affine Function that maps ALL lattice points to $2^a$

In [23]:
def common_prefix_length(label1, label2):
    common = 0
    for i in range(len(label1)):
        if label1[i] != label2[i]:
            break
        common += 1
    return common
#
def generate_pick_matrix(a):
    """
    Generates the Pick Matrix for all integers in generation 'a'.
    """
    integer_data = []
    target = 2**a
    
    # Iterate through all possible integer labels of length a
    for i in range(2**(a)):
        label = bin(i)[2:].zfill(a)[::-1] # LSB first representation
        T = mrTupFromPath(label)
        val = mrTupValue(T)
        if val[1] == 1:
            # We have an integer ...
            A, B = generationAffineParamsFromPath(label)
            
            # Check for integrality: (2^a - B) must be divisible by A
            if (target - B) % A == 0:
                n = (target - B) // A
                if n > 0:
                    # Validate the trajectory parity matches the label
                    curr, valid = n, True
                    for bit in label:
                        if (bit == '0' and curr % 2 == 0) or (bit == '1' and curr % 2 != 0):
                            valid = False; break
                        curr = (3*curr + 1)//2 if bit == '0' else curr // 2
                    
                    if valid:
                        integer_data.append({'n': n, 'label': label, 'slope': A / target})
    
        # Construct the Matrix P_ij = (1 - wi*wj) * 2^cp
        n_size = len(integer_data)
        P = np.zeros((n_size, n_size))
        for i in range(n_size):
            for j in range(n_size):
                wi, wj = integer_data[i]['slope'], integer_data[j]['slope']
                cp = common_prefix_length(integer_data[i]['label'], integer_data[j]['label'])
                P[i, j] = (1 - wi * wj) * (2**cp)
            
    return pd.DataFrame(integer_data), P



In [24]:
def presentPickMatric(P):
    pp = np.zeros(P.shape, dtype=int)
    for i in range(P.shape[0]):
        for j in range(P.shape[1]):
            pp[i, j] = math.ceil(math.log(P[i, j], 2))
    return pp
#

In [25]:
gen_num = 8
df_ints, pick_matrix = generate_pick_matrix(gen_num)

print(f"Generation {gen_num} Integer Solutions: {df_ints['n'].tolist()}")
print(f"Matrix Dimension: {pick_matrix.shape}")
print(f"Smallest Eigenvalue: {np.min(np.linalg.eigvalsh(pick_matrix))}")

Generation 8 Integer Solutions: [1, 4, 5, 16, 6, 20, 21, 64, 24, 26, 80, 84, 85, 256]
Matrix Dimension: (14, 14)
Smallest Eigenvalue: 191.84919298179895


In [26]:
presentPickMatric(pick_matrix)

array([[8, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0],
       [0, 8, 0, 2, 1, 4, 0, 2, 2, 1, 2, 4, 0, 2],
       [2, 0, 8, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0],
       [0, 2, 0, 8, 1, 2, 0, 4, 3, 1, 6, 2, 0, 4],
       [0, 1, 0, 1, 8, 1, 0, 1, 1, 2, 1, 1, 0, 1],
       [0, 4, 0, 2, 1, 8, 0, 2, 2, 1, 2, 6, 0, 2],
       [2, 0, 4, 0, 0, 0, 8, 0, 0, 0, 0, 0, 6, 0],
       [0, 2, 0, 4, 1, 2, 0, 8, 3, 1, 4, 2, 0, 6],
       [0, 2, 0, 3, 1, 2, 0, 3, 8, 1, 3, 2, 0, 3],
       [0, 1, 0, 1, 2, 1, 0, 1, 1, 8, 1, 1, 0, 1],
       [0, 2, 0, 6, 1, 2, 0, 4, 3, 1, 8, 2, 0, 4],
       [0, 4, 0, 2, 1, 6, 0, 2, 2, 1, 2, 8, 0, 2],
       [2, 0, 4, 0, 0, 0, 6, 0, 0, 0, 0, 0, 8, 0],
       [0, 2, 0, 4, 1, 2, 0, 6, 3, 1, 4, 2, 0, 8]])

In [27]:
gen_num = 9
df_ints, pick_matrix = generate_pick_matrix(gen_num)

print(f"Generation {gen_num} Integer Solutions: {df_ints['n'].tolist()}")
print(f"Matrix Dimension: {pick_matrix.shape}")
print(f"Smallest Eigenvalue: {np.min(np.linalg.eigvalsh(pick_matrix))}")

Generation 9 Integer Solutions: [2, 8, 3, 10, 32, 12, 13, 40, 42, 128, 48, 17, 52, 53, 160, 168, 170, 512]
Matrix Dimension: (18, 18)
Smallest Eigenvalue: 383.92465619426054


In [28]:
presentPickMatric(pick_matrix)

array([[9, 1, 0, 3, 1, 1, 0, 1, 3, 1, 1, 0, 1, 0, 1, 1, 3, 1],
       [1, 9, 0, 1, 3, 2, 0, 5, 1, 3, 3, 0, 2, 0, 3, 5, 1, 3],
       [0, 0, 9, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [3, 1, 0, 9, 1, 1, 0, 1, 5, 1, 1, 0, 1, 0, 1, 1, 5, 1],
       [1, 3, 0, 1, 9, 2, 0, 3, 1, 5, 4, 0, 2, 0, 7, 3, 1, 5],
       [1, 2, 0, 1, 2, 9, 0, 2, 1, 2, 2, 0, 3, 0, 2, 2, 1, 2],
       [0, 0, 1, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 0],
       [1, 5, 0, 1, 3, 2, 0, 9, 1, 3, 3, 0, 2, 0, 3, 7, 1, 3],
       [3, 1, 0, 5, 1, 1, 0, 1, 9, 1, 1, 0, 1, 0, 1, 1, 7, 1],
       [1, 3, 0, 1, 5, 2, 0, 3, 1, 9, 4, 0, 2, 0, 5, 3, 1, 7],
       [1, 3, 0, 1, 4, 2, 0, 3, 1, 4, 9, 0, 2, 0, 4, 3, 1, 4],
       [0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 9, 0, 2, 0, 0, 0, 0],
       [1, 2, 0, 1, 2, 3, 0, 2, 1, 2, 2, 0, 9, 0, 2, 2, 1, 2],
       [0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 9, 0, 0, 0, 0],
       [1, 3, 0, 1, 7, 2, 0, 3, 1, 5, 4, 0, 2, 0, 9, 3, 1, 5],
       [1, 5, 0, 1, 3, 2, 0, 7, 1, 3, 3, 0, 2, 0, 3, 9,

In [29]:
gen_num = 10
df_ints, pick_matrix = generate_pick_matrix(gen_num)

print(f"Generation {gen_num} Integer Solutions: {df_ints['n'].tolist()}")
print(f"Matrix Dimension: {pick_matrix.shape}")
print(f"Smallest Eigenvalue: {np.min(np.linalg.eigvalsh(pick_matrix))}")

Generation 10 Integer Solutions: [1, 4, 5, 16, 6, 20, 21, 64, 24, 26, 80, 84, 85, 256, 96, 11, 34, 104, 35, 106, 320, 336, 113, 340, 341, 1024]
Matrix Dimension: (26, 26)
Smallest Eigenvalue: 767.9623329089142


In [31]:
4 * 191.84919298179895 

767.3967719271958

In [30]:
presentPickMatric(pick_matrix)

array([[10,  0,  2,  0,  0,  0,  2,  0,  0,  0,  0,  0,  2,  0,  0,  1,
         0,  0,  1,  0,  0,  0,  4,  0,  2,  0],
       [ 0, 10,  0,  2,  1,  4,  0,  2,  2,  1,  2,  4,  0,  2,  2,  0,
         1,  2,  0,  1,  2,  2,  0,  4,  0,  2],
       [ 2,  0, 10,  0,  0,  0,  4,  0,  0,  0,  0,  0,  4,  0,  0,  1,
         0,  0,  1,  0,  0,  0,  2,  0,  4,  0],
       [ 0,  2,  0, 10,  1,  2,  0,  4,  3,  1,  6,  2,  0,  4,  4,  0,
         1,  3,  0,  1,  4,  6,  0,  2,  0,  4],
       [ 0,  1,  0,  1, 10,  1,  0,  1,  1,  2,  1,  1,  0,  1,  1,  0,
         2,  1,  0,  2,  1,  1,  0,  1,  0,  1],
       [ 0,  4,  0,  2,  1, 10,  0,  2,  2,  1,  2,  6,  0,  2,  2,  0,
         1,  2,  0,  1,  2,  2,  0,  6,  0,  2],
       [ 2,  0,  4,  0,  0,  0, 10,  0,  0,  0,  0,  0,  6,  0,  0,  1,
         0,  0,  1,  0,  0,  0,  2,  0,  6,  0],
       [ 0,  2,  0,  4,  1,  2,  0, 10,  3,  1,  4,  2,  0,  6,  5,  0,
         1,  3,  0,  1,  8,  4,  0,  2,  0,  6],
       [ 0,  2,  0,  3,  1,  2, 

In [6]:
gen_num = 11
df_ints, pick_matrix = generate_pick_matrix(gen_num)

print(f"Generation {gen_num} Integer Solutions: {df_ints['n'].tolist()}")
print(f"Matrix Dimension: {pick_matrix.shape}")
print(f"Smallest Eigenvalue: {np.min(np.linalg.eigvalsh(pick_matrix))}")

Generation 11 Integer Solutions: [192, 7, 22, 68, 69, 208, 23, 70, 212, 213, 640, 672, 75, 226, 680, 227, 682, 2048]
Matrix Dimension: (18, 18)
Smallest Eigenvalue: 1919.9976501284414


In [7]:
gen_num = 20
df_ints, pick_matrix = generate_pick_matrix(gen_num)

print(f"Generation {gen_num} Integer Solutions: {df_ints['n'].tolist()}")
print(f"Matrix Dimension: {pick_matrix.shape}")
print(f"Smallest Eigenvalue: {np.min(np.linalg.eigvalsh(pick_matrix))}")

Generation 20 Integer Solutions: [98304, 1152, 43, 130, 392, 131, 394, 1184, 396, 397, 1192, 1194, 3584, 132, 133, 400, 134, 404, 405, 1216, 408, 410, 1232, 1236, 1237, 3712, 3744, 1250, 3752, 1251, 3754, 11264, 11520, 1284, 1285, 3856, 1286, 3860, 3861, 11584, 3864, 3866, 11600, 11604, 11605, 34816, 35328, 11808, 3938, 11816, 3939, 11818, 35456, 1313, 3940, 3941, 11824, 3942, 11828, 11829, 35488, 35496, 35498, 106496, 3840, 1296, 433, 1300, 1301, 3904, 434, 1304, 435, 1306, 3920, 3924, 3925, 11776, 11904, 441, 1324, 1325, 3976, 3978, 11936, 1326, 3980, 3981, 11944, 11946, 35840, 36096, 12048, 4017, 12052, 12053, 36160, 1339, 4018, 12056, 4019, 12058, 36176, 36180, 36181, 108544, 109056, 36384, 4043, 12130, 36392, 12131, 36394, 109184, 12132, 12133, 36400, 12134, 36404, 36405, 109216, 109224, 109226, 327680, 344064, 38400, 4272, 1425, 4276, 4277, 12832, 12840, 12842, 38528, 475, 1426, 4280, 1427, 4282, 12848, 12852, 12853, 38560, 38568, 38570, 115712, 115968, 12888, 12890, 38672, 38676

# Smallest eigenvalue will always increase with each generation

In comparing the Pick Matrix for generation 8 with 10 we can see why the smallest eigenvalue will always increase:

The generation 8 matrix is embeded in the generation 10 matrix, but the diagonal values are 4X in generation 10 than 8

The values for the additions will on average always be larger than the values from the embedded predecessor.

In fact, we expect the smallest eigen value to increase ~4X every 2 generations.
